# The non-linear Kirchhoff model

## Model and energy

## The isometry constraint

## Discretisation

## Linearisation of the nodal isometry constraint

> For this, it is important ot realize that for the empleyed finite element space $W_h$, the nodal values of the discrete deformation ($u_h(z) : z \in \mathcal{N}_h$) are mutually independent variables in the minimization problem.

## Discrete $H^2$ gradient flow

# Implementation


Besides the work already done for [the linear case](linear-kirchhoff.ipynb), we need to compute the matrix $B_{k-1}$ which enforces the nodal isometry constraint for the solution $d_t y^k$ at step $k$.

The system to solve is (Bartels, 2013):

$$\left(\begin{array}{cc}
  (1 + \alpha \tau) M^{\top} A^{(2)} M & B^{\top}_{k - 1}\\
  B_{k - 1} & I
\end{array}\right)  \left(\begin{array}{c}
  d_t Y^k\\
  \Lambda
\end{array}\right) = \left(\begin{array}{c}
  - \alpha M^{\top} A^{(2)} MY^{k - 1} + \tau F\\
  0
\end{array}\right)$$

The local tensor matrix requires then the discrete gradient matrix $M$, the local tensor for the form $(\nabla u, \nabla v)$ in $P_2$ and the constraints matrix $B_{k-1}$. Note also that we now have vector valued functions and this will probably require tweaking some of the previous code.

## Discrete isometry constraint

\\[ \left( \begin{array}{ccccccccc}
     \quad & 2 y^{k - 1}_{i, 1} &  & \quad & 2 y^{k - 1}_{i, 1} &  & \quad & 2
     y^{k - 1}_{i, 1} & \\
     & y^{k - 1}_{i, 2} & y^{k - 1}_{i, 1} &  & y^{k - 1}_{i, 2} & y^{k -
     1}_{i, 1} &  & y^{k - 1}_{i, 2} & y^{k - 1}_{i, 1}\\
     & y^{k - 1}_{i, 2} & y^{k - 1}_{i, 1} &  & y^{k - 1}_{i, 2} & y^{k -
     1}_{i, 1} &  & y^{k - 1}_{i, 2} & y^{k - 1}_{i, 1}\\
     &  & 2 y^{k - 1}_{i, 2} &  &  & 2 y^{k - 1}_{i, 2} &  &  & 2 y^{k -
     1}_{i, 2}
   \end{array}\right)  \left(\begin{array}{c}
     d_t y^k_i\\
     d_t y^k_{i, 1}\\
     d_t y^k_{i, 2}\\
     d_t y^k_i\\
     d_t y^k_{i, 1}\\
     d_t y^k_{i, 2}\\
     d_t y^k_i\\
     d_t y^k_{i, 1}\\
     d_t y^k_{i, 2}
   \end{array}\right) \\]

# Tests

In [ ]:
from dolfin import *
import numpy as np
np.set_printoptions(precision=4, linewidth=130, threshold=5000, suppress=True)

import nbimporter
from interpolation import interpolate

import matplotlib.pyplot as pl
%matplotlib inline

#info(parameters, True)

def bitmap(A, rtol=1e-5, atol=1e-8, figsize=(10,10), cmap='binary', **kwargs):
    """Draw the number of non zeros of A."""
    if np.sum(A.shape) > 4000:
        print("Matrix is too big (%d x %d)" % A.shape)
    pl.figure(figsize=figsize)
    bmap = 1 - np.isclose(A, 0, rtol, atol).astype(np.int)
    pl.imshow(bmap, cmap=cmap, **kwargs)
    print("%.2f%% non zeros" % (100*bmap.sum()/np.product(bmap.shape)))

In [ ]:
#path = "/home/fenics/local/src/nonlinear-kirchhoff/build"
path = "/home/fenics/.netbeans/remote/127.0.0.1/hpel620-Windows-x86_64/C/Users/Miguel/Devel/nonlinear-kirchhoff"
A = np.loadtxt(path + "/A.data")
Bk = np.loadtxt(path + "/Bk.data")
#D = np.loadtxt(path + "/D.data")
D = np.identity(Bk.shape[0])
Fk = np.loadtxt(path + "/Fk.data")
Y0 = np.loadtxt(path + "/y0.data")
Yk = np.loadtxt(path + "/yk.data")
dtY = np.loadtxt(path + "/dtY_L.data")
#P26 = np.loadtxt(path + "/P26.data")
# Cut out the bottom extra chunk in the solution and RHS
dtY = dtY[:-Bk.shape[0]]
Fk = Fk[:-Bk.shape[0]]

In [ ]:
A.shape, Bk.shape, Fk.shape, Y0.shape, Yk.shape, dtY.shape #, P26.shape

In [ ]:
E = np.loadtxt(path+"/energy.data")
pl.figure(figsize=(16,4))
_ = pl.plot(E)

## Exploring the stiffness matrix

First a nice picture:

In [ ]:
bitmap(A, figsize=(5,5), interpolation='none')

The condition nunmber is **awful**, something is definitely wrong:

In [ ]:
np.linalg.cond(A)

In [ ]:
bitmap(Bk[176:180,190:200], figsize=(10,4), interpolation='none')
print(Bk[176:180,190:200])

In [ ]:
W = VectorFunctionSpace(UnitSquareMesh(10,10, "right"), "DKT", degree=3, dim=3)
v2d = vertex_to_dof_map(W)
d2v = (dof_to_vertex_map(W)/W.dofmap().num_entity_dofs(0)).astype(np.int)
print((d2v.min(),d2v.max()))

#idx = 44
#for sub in range(3):
#    print(v2d[9*idx + 3*sub + 0], v2d[9*idx + 3*sub + 1], v2d[9*idx + 3*sub + 2])

#A[189:197,189:197], Fk[189:197], dtY[189:197]

In [ ]:
# reread full RHS and solve here
Fk = np.loadtxt(path + "/Fk.data")
dtY = np.loadtxt(path + "/dtY_L.data")

assert np.all((D - np.identity(D.shape[0])) == 0), "Padding matrix must be the identity"
M = np.block([[A, Bk.T],[Bk, D]])

# Compare np's solution with ours
dZ = np.linalg.solve(M,Fk)
np.allclose(dZ, dtY)

## Visualizing the components of the solution

A quick visualization skipping Paraview...

Recall that the $W^3$ space has 27 dofs in 3 groups of 9, one per subspace. Inside each group the dofs are: [evaluation, eval of dx, eval of dy] at each of the 3 vertices in turn.

In [ ]:
W = VectorFunctionSpace(RectangleMesh(Point(-2,0), Point(2,1), 10, 10, "right"),
                        "DKT", degree=3, dim=3)
y0 = Function(W)
yk = Function(W)

In [ ]:
Y0 = np.loadtxt(path + "/y0.data")
Yk = np.loadtxt(path + "/yk.data")
y0.vector().set_local(Y0)
_ = yk.vector().set_local(Yk)

In [ ]:
_xx = np.array(sorted(W.mesh().coordinates()[:,0]))
#_xx = np.arange(-2,-1.8,0.001)
pl.figure(figsize=(12,6))
for _y in np.linspace(0,1,6):
    pl.plot(_xx, [y0(x,_y)[2] - 0 for x in _xx], label='$u_3^0(%.1f)$' % _y)
for _y in np.linspace(0,1,6):
    pl.plot(_xx, [yk(x,_y)[2] - 0. for x in _xx], label='$u_3^k(%.1f)$' % _y)
_ = pl.legend()

For later use, let us extract the dofs for evaluations and both partial derivatives and arrange them into arrays with three components, one per dimension of the range.

In [ ]:
from dofs import plot_dofs, plot_field_at_dofs, extract_dofs_with_mask

_f = extract_dofs_with_mask(W, np.arange(0,27,3))
f = [_f[::3], _f[1::3], _f[2::3]]
_dfdx = extract_dofs_with_mask(W, np.arange(1,27,3))
dfdx = [_dfdx[::3], _dfdx[1::3], _dfdx[2::3]]
_dfdy = extract_dofs_with_mask(W, np.arange(2,27,3))
dfdy = [_dfdy[::3], _dfdy[1::3], _dfdy[2::3]]

In [ ]:
_sf, _sdfdx, _sdfdy = set(_f), set(_dfdx), set(_dfdy)

The initial condition has to have $\nabla^T y^0 \nabla y^0 = I_2$ at all nodes. If the following is not `(True, True, True)`, then we have a(nother) problem.

In [ ]:
nvert = W.mesh().num_vertices()
dxdx = np.dot(Y0[_dfdx], Y0[_dfdx]) / nvert
dxdy = np.dot(Y0[_dfdx], Y0[_dfdy]) / nvert
dydy = np.dot(Y0[_dfdy], Y0[_dfdy]) / nvert
np.isclose(dxdx, 1), np.isclose(dxdy, 0), np.isclose(dydy,1)

In [ ]:
dxdx, dxdy, dydy

How is it possible that dxdy is not 0? $\partial_x y_0$ and $\partial_y y_0$ should be orthogonal!

All iterates $y^k$ fulfill the boundary condition since updates are always made with $d_tY$ with zero value and derivative at the boundary.

In [ ]:
np.allclose(Yk[dfdx[0]], 1), np.allclose(Yk[dfdy[1]], 1)

That was bad! Let's see where it failed

In [ ]:
np.where(np.logical_not(np.isclose(Y0[dfdx[0]], 1.)))[0].size

In [ ]:
pl.figure(figsize=(16,6))
pl.subplot(2,2,1)
plot_dofs(W, np.where(np.logical_not(np.isclose(Y0[dfdy[0]], 0.)))[0])
pl.subplot(2,2,2)
plot_dofs(W, np.where(np.logical_not(np.isclose(Y0[dfdy[1]], 1.)))[0])
pl.subplot(2,2,3)
plot_dofs(W, np.where(np.logical_not(np.isclose(Y0[dfdy[2]], 0.)))[0])

#plot_dofs(W, np.where(np.logical_not(np.isclose(Yk[dfdx[0]], 1.)))[0])
#pl.subplot(2,2,4)
#plot_dofs(W, np.where(np.logical_not(np.isclose(Yk[dfdy[1]], 1.)))[0])

Something is clearly off, let's plot the components of the solution:

In [ ]:
def plot_component(Y, W, i, ii, **kwargs):
    y = Function(FunctionSpace(W.mesh(), "DKT", 3))
    vec = np.zeros_like(y.vector().array())
    vec[i::3] = Yk[ii[i]]/1e8
    _ = y.vector().set_local(vec)

    P = FunctionSpace(W.mesh(), "Lagrange", 1)
    p = Function(P)
    vals = y.compute_vertex_values(W.mesh())

    if np.any(np.isnan(vals)):
        print("%.2f%% NaNs in vertex values for component %d, unable to plot"
              % (100*np.sum(np.isnan(vals))/len(vals), i))
        return
    d2v = dof_to_vertex_map(P)
    _ = p.vector().set_local(vals[d2v])

    plot(p, **kwargs)

In [ ]:
plot_component(dtY, W, 0, f)

In [ ]:
pl.figure(figsize=(14,4))
pl.subplot(1,2,1)
plot_component(Y0, W, 1, dfdx)
pl.subplot(1,2,2)
plot_component(Y0, W, 2, dfdy)

In [ ]:
pl.figure(figsize=(18,10))
pl.subplot(3,3,1)
plot_component(Yk, W, 0, f, title='$y_k^0$')
pl.subplot(3,3,2)
plot_component(Yk, W, 1, f, title='$y_k^1$')
pl.subplot(3,3,3)
plot_component(Yk, W, 2, f, title='$y_k^2$')
pl.subplot(3,3,4)
plot_component(Yk, W, 0, dfdx, title='$\partial_x{y_k^0}$')
pl.subplot(3,3,5)
plot_component(Yk, W, 1, dfdx, title='$\partial_x{y_k^1}$')
pl.subplot(3,3,6)
plot_component(Yk, W, 2, dfdx, title='$\partial_x{y_k^2}$')
pl.subplot(3,3,7)
plot_component(Yk, W, 0, dfdy, title='$\partial_y{y_k^0}$')
pl.subplot(3,3,8)
plot_component(Yk, W, 1, dfdy, title='$\partial_y{y_k^1}$')
pl.subplot(3,3,9)
plot_component(Yk, W, 2, dfdy, title='$\partial_y{y_k^2}$')

## Inspecting the $P_2^{3 \times 2}$ tensor

Plots of the two local tensors for a 1x1 grid. What are those spurious entries on the top right corner?

In [ ]:
bitmap(P26, figsize=(5,5), cmap='Purples', interpolation='none')

```c++
  // position in array is destination row, value is source row:
  int permutations[] = {0,6,1,7,2,8,3,9,4,10,5,11};
```

In [ ]:
T = TensorFunctionSpace(UnitSquareMesh(1,1), "Lagrange", 2, shape=(3,2))
for sub in range(6):
    print(T.sub(sub).dofmap().cell_dofs(0))

dm = T.dofmap()
dm.cell_dofs(0)

This replicates the contents of the UFL file:

In [ ]:
domain = W.mesh()
T = TensorFunctionSpace(domain, "Lagrange", 2, shape=(3,2))
#W = VectorFunctionSpace(domain, "DKT", 3, dim=3)
P = VectorFunctionSpace(domain, "Lagrange", 3, dim=3)

p = TrialFunction(T)
q = TestFunction(T)
p22 = inner(nabla_grad(p), nabla_grad(q))*dx

u = TrialFunction(W)
v = TestFunction(W)
dkt = inner(u,v)*dx

f = Coefficient(W)
force = inner(f,v)*dx

# Define variational problem for projection
g = Coefficient(P)
Pg = TrialFunction(W)
w = TestFunction(W)
project_lhs = inner(w, Pg)*dx
project_rhs = inner(w, g)*dx

In [ ]:
A = assemble(p22)
A.array().shape

In [ ]:
bitmap(A.array())

### The local tensor on a 1x1 grid

Plots of the two local tensors for a 1x1 grid. What are those spurious entries on the top right corner?

In [ ]:
Alocbmap = (np.round(Aloc, 5) != 0).astype(np.int)
pl.imshow(Alocbmap)

In [ ]:
Aloc2bmap = (np.round(Aloc2, 5) != 0).astype(np.int)
pl.imshow(Aloc2bmap)

In [ ]:
v = 2  # m/s
r = 0.152 / 2  # m
A = np.pi*r**2

In [ ]:
LN = 10 + 50 * np.log (v) + 10 * np.log (A) 

In [ ]:
LN

In [ ]:
W = VectorFunctionSpace(UnitSquareMesh(1,1, "right"), "DKT", degree=3, dim=3)
T = TensorFunctionSpace(W.mesh(), "Lagrange", degree=2, shape=(3,2))

In [ ]:
msh = W.mesh()
geo = msh.geometry()

In [ ]:
for i in range(3):
    dm = W.sub(i).dofmap()
    print(dm.cell_dofs(0))

In [ ]:
for i in range(3):
    dm = W.sub(i).dofmap()
    print(dm.cell_dofs(1))

In [ ]:
for i in range(3):
    dm = T.sub(i).dofmap()
    print(dm.cell_dofs(0))

In [ ]:
for i in range(3):
    dm = T.sub(i).dofmap()
    print(dm.cell_dofs(1))

# Tests

In [ ]:
from dolfin import *
msh = RectangleMesh(Point(-2,0), Point(2,1), 400, 20, "crossed")
V = FunctionSpace(msh, "Lagrange", 1)
v = TestFunction(V)
b = assemble(v*dx)

In [ ]:
b.array().shape

In [ ]:
W = VectorFunctionSpace(msh, "DKT", degree=3, dim=3)
w = TestFunction(W)

p = Constant((1,0,0))

a = assemble(inner(v, p)*dx)

#c = assemble(inner(w,p)*dx)

a,b #,c

In [ ]:
import numpy as np
np.all(a.array()==b.array())

In [ ]:
set(a.array()[:333:3].round(6))